# T5 fine-tuning

In this notebook I tried to do fine-tuning for T5

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from transformers import TrainingArguments, Trainer
import pandas as pd
import numpy as np

## Data Loading

In [2]:
# Load the toxic-to-non-toxic dataset from CSV files
train_data = pd.read_csv('../data/interim/train.csv')
test_data = pd.read_csv('../data/interim/test.csv')

In [11]:
train_data = train_data[:1000]
test_data = test_data[:500]

## Model and Tokenizer Loading 

In [5]:
# Initialize the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Create datasets for model

In [4]:
# Define a custom dataset class
class TextDetoxDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=512, max_target_length=128, text_column="source", target_column="target"):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length
        self.text_column = text_column
        self.target_column = target_column

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source_text = str(self.data.iloc[idx][self.text_column])
        target_text = str(self.data.iloc[idx][self.target_column])

        source_tokenized = self.tokenizer(
            source_text, truncation=True, padding="max_length", max_length=self.max_input_length, return_tensors="pt"
        )

        target_tokenized = self.tokenizer(
            target_text, truncation=True, padding="max_length", max_length=self.max_target_length, return_tensors="pt"
        )

        return {
            "input_ids": source_tokenized["input_ids"].squeeze(),
            "attention_mask": source_tokenized["attention_mask"].squeeze(),
            "labels": target_tokenized["input_ids"].squeeze(),
        }

In [12]:
# Create datasets
train_dataset = TextDetoxDataset(train_data, tokenizer)
test_dataset = TextDetoxDataset(test_data, tokenizer)

## Fine-tune the model

In [13]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./text_detox_model",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs",
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=200,
)

In [14]:
# Define data collator
data_collator = lambda data: {
    "input_ids": torch.stack([item["input_ids"] for item in data]),
    "attention_mask": torch.stack([item["attention_mask"] for item in data]),
    "labels": torch.stack([item["labels"] for item in data]),
}

In [15]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
#     eval_dataset=test_dataset,
)


In [16]:
# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [ ]:
# Save the fine-tuned model
model.save_pretrained("./text_detox_model")